# Group 10: Title 

### Background information:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v

### Our research question:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec et risus pulvinar, viverra neque at, interdum tortor.

### The dataset:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v



## Preliminary Exploratory Data Analysis

What we plan to summarize:

What we plan to plot:

In [1]:
# Retrieving nessesary packages
library(dplyr)
library(tidyverse)
library(rvest)
library(tidymodels)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.

In [23]:
# Reading the data file
url = "https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS"

player_data = read_csv(url)

processed_player_data = player_data |>
                        separate(Age, c("Age", "Birth date"), sep = " ") |>
                        seperate(Current_Rank, c())

processed_player_data

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: Error in FUN(left, right): operations are possible only for numeric, logical or complex types


In [ ]:
# Summary for data
data_summary = data

data_summary

In [ ]:
# Visualization for data
options(repr.plot.width=20, repr.plot.height=20)

data_plot = data |>
            ggplot(aes(x=..,y=..)) +
            geom_point() +
            xlab("x") +
            ylab("y") +
            ggTitle("x vs y") +
            theme(text = element_text(size=20))

data_plot

## Methods

### Relevant variables:
- X
- Y

### Outlined procedure:
1. one
2. two
3. three
4. four
5. five

### Visualization:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v

## Expected Outcomes and Significance

### Expected findings:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v

### Relevancy of findings:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v

### Future studies:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v